In [128]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [129]:
def read_images(image_path):
    img = cv2.imread(image_path)
    img  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

In [130]:
def create_compute_shift(img):
    shift_detector = cv2.xfeatures2d.SIFT_create()
    kp, desc = shift_detector.detectAndCompute(img, None)
    return kp, desc

In [131]:
def get_distance(desc1, desc2):
    dist = scipy.spatial.distance.cdist(desc1, desc2, 'sqeuclidean')
    return dist

In [132]:
left_img = read_images('./images/left.jpg')
right_img = read_images('./images/right.jpg')
left_key_point, left_descriptor = create_compute_shift(left_img)
right_key_point, right_descriptor = create_compute_shift(right_img)

In [133]:
distance = get_distance(left_descriptor, right_descriptor)

In [134]:
x, y = np.where(distance < 7000)

In [140]:
x

array([3563, 3574, 3575, 3581, 3604, 3616, 3621, 3738, 3739, 3750, 3817,
       3824, 3835, 3928, 3969, 3976, 4197, 4197, 4200, 4201, 4202, 4254,
       4254, 4295, 4312, 4321, 4325, 4327, 4356, 4360, 4382])

In [136]:
c = np.concatenate((a,b), axis=1)

In [137]:
c

array([[2, 3, 1, 1],
       [4, 6, 1, 1]])